In [ ]:

class Model():
    def __init__(self,pumpsize,pumpmodel,attribute_dict):
        self.pumpSize=pumpsize #adding hydraulic model name
        self.pumpModel=pumpmodel
        self.attribute_dict=attribute_dict # Lets make an attribute dict that has the relevant attributes for this model
        self.CurveGroups=[]
        self.name=self.__repr__()
        self.raw_hp = 0

    def __repr__(self):
        return f"{self.pumpModel} {self.pumpSize}"

    def add_to_CurveGroups(self,curve):
        """This will add a CurveGroup object to the dict"""
        self.CurveGroups.append(curve)
      
class CurveGroup():
    def __init__(self,curve_number):
        self.CurveList=[] #Lets add a list of curves
        self.PLO_dict={} #Adds a PLO dict for the options that are used for this Curve Groups 
        self.CurveList=[] #Lets add a list of curve objects into this list  
        self.curve_number=curve_number #Lets have the parent curve number
    
    def __repr__(self):
        return self.curve_number

class Curve():
    """
    Example:
    CurveType
        X           Y
        [1,2,3]     [1,2,3]
    Poles: Number of Poles the Pump is running at i.e 2,4,6, etc. 
    Trim: Trim of the curve
    HP: End of Curve HP
    """
    CONST=(9810/(3.6*(10**6)))
    KW_to_HP=0.746

    def __init__(self,poles,trim,parent,curve_xpath):
        self.poles=poles #What is the HP MAX
        # self.curve_dict={'Head':{'Coeff':[],'XY-Points':{'x':[],'y':[]}},'NPSH':{'Coeff':[],'XY-Points':{'x':[],'y':[]}},'Efficiency':{'Coeff':[],'XY-Points':{'x':[],'y':[]}},'Power':{'Coeff':[],'XY-Points':{'x':[],'y':[]}}}
        self.curve_dict={'Head':{'Coeff':[],'XY-Points':[]},'NPSH':{'Coeff':[],'XY-Points':[]},'Efficiency':{'Coeff':[],'XY-Points':[]},'Power':{'Coeff':[],'XY-Points':[]}}
        self.xpath_dict=curve_xpath
        self.trim=trim #What is the trim on these curves
        self.parent=parent
        # self.HP='N/A'
        self.name=self.__repr__()

    def __repr__(self):
        return f"{self.parent}/{self.trim}"
    
    @classmethod
    def change_CONSTANT(cls,value):
        cls.CONST=value
    
    @classmethod
    def change_CONVERSION_FACTOR(cls,value):
        cls.KW_to_HP=value
    
    def _round_to_HPStep(self,HP,tol=.5):
        """Takes a calculated HP and rounds it to the correct motor step."""
        STEPS = [0.5, 0.75, 1, 1.5, 2, 3, 5, 7.5, 10, 15, 20, 25, 30, 40, 50, 60, 75, 100, 125, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 1250, 1500, 1750, 2000]
        for step in STEPS:
            if step >= HP or abs(HP-step)<=tol:
                return step
    
    def get_xy_points(self,CurveType,xml_ele):
        #Lets get the x-y points
        points=[]
        for curve in xml_ele.find(f"{self.xpath_dict[CurveType]}"):
            points.append((curve.find("./x").text,curve.find("./y").text))
        return points

    def return_xy_points_list(self,CurveType:str,return_val:str):
        """This function will return the x-points as a list from the XY-Points. You can either return x, y, or both points into lists using this."""
        
        if CurveType not in self.curve_dict.keys():
            raise ValueError(f"You must choose from one of the following: {','.join(self.curve_dict.keys())}")
        
        if not self.curve_dict[CurveType]['XY-Points']: #Do we have an actual list
            raise ValueError('No points in the dictionary')
        
        points=self.curve_dict[CurveType]['XY-Points']
        x,y=zip(*points)
        x=[float(i) for i in x]
        y=[float(i) for i in y]

        if return_val.strip().lower()=='x':
            return x
        
        elif return_val.strip().lower()=='y':
            return y
        
        elif return_val.strip().lower()=='both':
            return x,y
        
        else:
            raise Exception('You must choose either x, y, or both for the return val')
        
    def getcurvepolynomial(self,deg,CurveType,xml_ele):
        """This returns the polynomial coefficients from the """
        points=self.get_xy_points(CurveType,xml_ele) #This returns a tuple
        # To change the points into x,y data
        x,y=zip(*points)
        x=[float(i) for i in x]
        y=[float(i) for i in y]
        coef=np.polyfit(x,y,deg)
        return coef
    

    def _find_max_HP_QH(self,CurveType,rounded_places:int=2):
        max_value=0
        for point in self.curve_dict[CurveType]['XY-Points']:
            x,y=point
            val=float(x)*float(y)
            #Check if x, y are floats 
            #Check if x,y are positive
            if val > max_value:
                max_value=val
        p_kw=self.CONST*max_value
        p_hp=round(p_kw/self.KW_to_HP,rounded_places)
        return p_hp


    def calculate_p_kw_points(self):
        list_power_tuples = []
        for flow,head in self.curve_dict['Head']['XY-Points']: #Assume flow and head in metric?
            p_kw = float(flow) * float(head) * self.CONST
            list_power_tuples.append((flow, str(p_kw)))
        return list_power_tuples
    

    def calculate_p_hp():
        pass


    def complete_curve_dict(self,xml_ele):
        for k,v in self.curve_dict.items():
            try:
                self.curve_dict[k]['Coeff']=self.getcurvepolynomial(5,k,xml_ele)
                self.curve_dict[k]['XY-Points']=self.get_xy_points(k,xml_ele)
                if k=='Head':
                    p_hp=self._find_max_HP_QH(k)
                    self.HP=self._round_to_HPStep(p_hp,tol=0.1)
            except:
                print(f"{k} does not exist in the XML") #add to a logger
                if k=='Power':
                    print(f"Calculating {k} values") #add to a logger
                    self.curve_dict[k]['XY-Points'] = self.calculate_p_kw_points()
    
    @property
    def min_flow(self):
        try:
            if not self.curve_dict['Head']['XY-Points']: #Do we have an actual list
                raise ValueError('No points in the dictionary')
            else:
                x_points=self.return_xy_points_list('Head','x')
                res=min(x_points)
                return res
        except TypeError:
            print(f"This data is of type {type(x_points[0])} and it needs to be a float or int.")
            return None

    @property
    def max_flow(self):
        try:
            if not self.curve_dict['Head']['XY-Points']: #Do we have an actual list
                raise ValueError('No points in the dictionary')
            else:
                x_points=self.return_xy_points_list('Head','x')
                res=max(x_points)
                return res
        except TypeError:
            print(f"This data is of type {type(x_points[0])} and it needs to be a float or int.")
            return None